In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA



import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim


In [ ]:
# Load data
X_male = np.load("Unify_Gait_Cycles/X_male.npy")
X_female = np.load("Unify_Gait_Cycles/X_female.npy")
label_male = [0]*X_male.shape[0]
label_female = [1]*X_female.shape[0]
num_male = X_male.shape[0]
num_female = X_female.shape[0]
gait_cycles = np.append(X_male, X_female, axis=0)
labels = np.append(label_male, label_female, axis = 0)

print(gait_cycles.shape)
print(labels.shape)

In [ ]:
gait_cycles_acc = gait_cycles[:,:,3]
poly_degree = 60

indices = np.linspace(0,100,100,endpoint=False)
indices = indices.T
coeffs_list = []
fits_list = []
for i in range(gait_cycles.shape[0]):
    cycle = gait_cycles_acc[i,:]
    # NOTE Full=True to suppress warning that doesn't seem to be important
    coeffs = np.polyfit(indices, cycle, poly_degree, full=True)[0]
    poly_fit = np.poly1d(coeffs)
    coeffs_list.append(coeffs)
    fits_list.append(poly_fit)



    


In [ ]:
num_clusters = 20
"""
for i in range(10):
    plt.plot(indices, fits_list[i](indices))
    plt.plot(indices, gait_cycles[i,:,3])
    plt.show()
"""
kmeans_results = KMeans(n_clusters=num_clusters, random_state=0, n_init=poly_degree+1).fit(coeffs_list)
# mult_vals = X@(kmeans.cluster_centers_[1].T)
# print(mult_vals)

print("Starting final for loop")
for i in range(num_clusters):
    male_count = 0
    female_count = 0
    cluster = np.where(i == kmeans_results.labels_)[0]
    centroid_fit = np.poly1d(kmeans_results.cluster_centers_[i].T)
    print(len(cluster))
    for j in range(len(cluster)):
        if cluster[j] < num_male:
            male_count += 1
            color = 'r'
            #plt.plot(range(0,100),gait_cycles[cluster[j],:,3], 'r--')
        else:
            female_count += 1
            color = 'b'
            #plt.plot(range(0,100),gait_cycles[cluster[j],:,3], 'b--')
        if j % 10 == 0:
            plt.plot(range(0,100),gait_cycles[cluster[j],:,3], color + '--')
    print("Num Males: ", male_count)
    print("Num Females: ", female_count)
    plt.plot(indices, centroid_fit(indices), 'g-')
    plt.show()
        

In [ ]:
# Shuffle the data
perm = np.random.permutation(gait_cycles.shape[0])
gait_cycles = gait_cycles[perm]
labels = labels[perm]




In [ ]:
gait_ind_cycles = gait_cycles
num_measurements = gait_ind_cycles.shape[1]
ax_bar = np.sum(gait_ind_cycles[:,:,0],axis=1)/num_measurements
ay_bar = np.sum(gait_ind_cycles[:,:,1],axis=1)/num_measurements
az_bar = np.sum(gait_ind_cycles[:,:,2],axis=1)/num_measurements

# Gravity vector
rho_list = np.array([ax_bar, ay_bar, az_bar]).T

# Normalized gravity vector 
zeta = rho_list / np.reshape(np.repeat(np.linalg.norm(rho_list, axis=1), 3, axis=0), (9968, 3))

# Acceleration matrix
A = gait_ind_cycles[:,:,0:3]

# Finding acceleration in up down direction
a_zeta = []
for i in range(zeta.shape[0]):
    sample_a_zeta = []
    for row in A[i]:
        A_g = np.dot(row,zeta[i])
        sample_a_zeta.append(A_g)
    a_zeta.append(sample_a_zeta)
a_zeta = np.array(a_zeta)
print(a_zeta.shape)




In [ ]:
# Find acceleration in flattened plane (without up down)
zeta_repeat = np.repeat(zeta, 100, axis=0)
zeta_repeat = np.reshape(zeta_repeat, (9968, 100, 3))
a_zeta_repeat = np.repeat(a_zeta, 3, axis=1)
a_zeta_repeat = np.reshape(a_zeta_repeat, (-1, 100,3))

A_f = A - zeta_repeat*a_zeta_repeat


In [ ]:
# Using PCA to find first principle direction (forward and back)
xi = []
for person in A_f:
    pca = PCA(n_components=1)
    pca.fit(person)
    xi.append(pca.components_)
xi = np.array(xi)
xi = np.reshape(xi, (-1,3))

# Extract acceleration in the xi direction
a_xi = []
for i in range(xi.shape[0]):
    sample_a_xi = []
    for row in A_f[i]:
        A_xi = np.dot(row,xi[i])
        sample_a_xi.append(A_xi)
    a_xi.append(sample_a_xi)
a_xi = np.array(a_xi)
print(a_xi.shape)


In [ ]:
# Use cross product to find third direction (right and left)
psi = np.cross(zeta,xi, axis=1)
print(psi.shape)

# Extract acceleration in the psi direction
a_psi = []
for i in range(psi.shape[0]):
    sample_a_psi = []
    for row in A_f[i]:
        A_psi = np.dot(row,psi[i])
        sample_a_psi.append(A_psi)
    a_psi.append(sample_a_psi)
a_psi = np.array(a_psi)
print(a_psi.shape)


In [ ]:
# Make new ndarray with new coordinate system

A_transformed = np.stack((a_zeta, a_xi, a_psi), axis = 2)
A_trans_mag = np.linalg.norm(A_transformed, axis=2)
A_transformed = np.stack((a_zeta, a_xi, a_psi,A_trans_mag), axis = 2)

#to_show = 6
to_show = 8
#to_show = 11
#to_show = 20
plt.figure(1)
plt.subplot(211)
plt.plot(range(0,100), A_transformed[to_show,:,0], color='orange')
plt.ylabel('zeta')
plt.subplot(212)
plt.plot(range(0,100), A_transformed[to_show,:,1], color='orange')
plt.ylabel('xi')
plt.figure(2)
plt.subplot(211)
plt.plot(range(0,100), A_transformed[to_show,:,2], color='orange')
plt.ylabel('psi')
plt.subplot(212)
plt.plot(range(0,100), A_transformed[to_show,:,3], color='orange')
plt.ylabel('magnitude')

plt.figure()
plt.plot(range(0,100), A_transformed[to_show,:,0], color='orange')
plt.title("zeta")
plt.figure()
plt.plot(range(0,100), A_transformed[to_show,:,1], color='orange')
plt.title("xi")
plt.figure()
plt.plot(range(0,100), A_transformed[to_show,:,2], color='orange')
plt.title("psi")
plt.figure()
plt.plot(range(0,100), A_transformed[to_show,:,3], color='orange')
plt.title("Magnitude")
plt.show()

In [ ]:
# gait_cycles_x = gait_cycles[:,:,0]

poly = preprocessing.PolynomialFeatures(10)
indices = np.linspace(0,100,100, endpoint=False)
indices = indices.T
X = poly.fit_transform(indices.reshape(-1,1))
print(X.shape)
lm_array = np.ndarray((9968,11))

lm_object_array = []
for i in range(gait_cycles.shape[0]):
    cycle = gait_cycles[i,:,3]     
    lm = linear_model.LinearRegression(fit_intercept=True)
    model = lm.fit(X,cycle)
    lm_array[i,:] = model.coef_
    lm_object_array.append(model)
#     plt.plot(indices,model.predict(X))
#     plt.plot(indices, gait_cycles[i,:,3])
#     plt.show()

# print(lm_array.shape)
# kmeans = KMeans(n_clusters=20, random_state=0, n_init=20).fit(lm_array)
# print(kmeans.labels_)

# inertia_array = []
# for i in range(1,15):
#     kmeans_i = kmeans = KMeans(n_clusters=i, random_state=0, n_init=20).fit(lm_array)
#     inertia_array.append(kmeans_i.inertia_)
# plt.plot(range(1,15),inertia_array)
# plt.show()


kmeans = KMeans(n_clusters=10, random_state=0, n_init=20).fit(lm_array)
# for i in range(10):
#     plt.plot(indices, X@lm_array[i].T)
#     plt.plot(indices, gait_cycles[i,:,3])
#     plt.show()

# mult_vals = X@(kmeans.cluster_centers_[1].T)
# print(mult_vals)

print("Starting final for loop")
for i in range(10):
    cluster = np.where(i == kmeans.labels_)[0]
    plot_vals = X@(kmeans.cluster_centers_[i].T)
    plt.plot(indices, plot_vals)
    plt.show()
    for j in cluster:
        
        plt.plot(range(0,100),gait_cycles[j,:,3], 'b-')
#         plt.plot(range(0,100), mean_array)
        
    plt.show()
    
   
        

In [ ]:
#num_validate = 1004
num_validate = 500
gait_cycles_train = gait_cycles[0:-num_validate]
gait_cycles_test = gait_cycles[-num_validate:]

# gait_cycles_train = A_transformed[0:-num_validate]
# gait_cycles_test = A_transformed[-num_validate:]

labels_train = labels[0:-num_validate]
labels_test = labels[-num_validate:]
# Make them nice tensors :)
batch_size = 9
# gait_cycles_train = torch.FloatTensor(gait_cycles_train[:,:,3]).reshape(-1,batch_size,1,100)
# gait_cycles_test = torch.FloatTensor(gait_cycles_test[:,:,3]).reshape(-1,1,1,100)
# labels_train = torch.LongTensor(labels_train).reshape(-1,batch_size)
# labels_test = torch.LongTensor(labels_test).reshape(-1, 1)
gait_cycles_train = torch.FloatTensor(gait_cycles_train).permute(0,2,1).reshape(1052,batch_size,4,100)
gait_cycles_test = torch.FloatTensor(gait_cycles_test).permute(0,2,1).reshape(500,1,4,100)
labels_train = torch.LongTensor(labels_train).reshape(-1,batch_size)
labels_test = torch.LongTensor(labels_test).reshape(-1, 1)

In [ ]:
class Flatten(nn.Module):
    
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
class Gait_Classifier(nn.Module):
    """
    TODO: Add attention
    """
    def __init__(self, input_size=100, kernel_size=10, hidden_size=500):
        super(Gait_Classifier, self).__init__()
        
        self.layers = nn.ModuleList()
        self.lstm_encoder = nn.LSTM(100, 100,2)
       # self.layers.append(nn.Conv1d(1, 20, kernel_size))
        self.layers.append(nn.Sequential(
            nn.Conv1d(4, 25, kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)))
        self.layers.append(nn.Sequential(
            nn.Conv1d(25, 50, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)))
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(Flatten())
        self.layers.append(nn.Linear(25*18, hidden_size))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(hidden_size, 2))
        self.layers.append(nn.Softmax(dim=1))
        
        
    def forward(self, x):
        #out = self.lstm_encoder(x)[0]
        out = x
        for layer in self.layers:
            out = layer(out)
        return out
    """
    def __init__(self, input_size=100, kernel_size=10, hidden_size=500):
        super(Gait_Classifier, self).__init__()
        
        self.layers = nn.ModuleList()
        self.lstm_encoder = nn.LSTM(100, 20)
       # self.layers.append(nn.Conv1d(1, 20, kernel_size))
        self.layers.append(nn.Sequential(
            nn.Conv1d(4, 20, kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)))
        self.layers.append(nn.Sequential(
            nn.Conv1d(20, 40, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)))
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(Flatten())
        self.layers.append(nn.Linear(20*18, hidden_size))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(hidden_size, 2))
        self.layers.append(nn.Softmax(dim=1))
        
        
    def forward(self, x):
        print(x.shape)
        out = self.lstm_encoder(x)
        for layer in self.layers:
            out = layer(out)
        return out

    """

In [ ]:
#https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/ Helped a lot!

def train_model(model, optimizer, x_train, y_train, x_test, y_test, batch_size, num_epochs=20):
    loss_list = []
    acc_list = []
    total_step = len(x_train)
    criterion = nn.CrossEntropyLoss()
    
    correct = 0.0
    model = model.eval()
    validation_loss = 0
    for (test_gait, test_lab) in zip(x_test, y_test):
        prediction = model(test_gait)
        validation_loss += criterion(prediction, test_lab)
        if prediction.argmax() == test_lab:
            correct += 1.0
    acc = correct / y_test.size()[0]
    acc_list.append(acc)
    validation_loss /= x_test.shape[0]
    loss_list.append(validation_loss)
    model = model.train()
    
    for epoch in range(num_epochs):
        for i, (gaits, labels) in enumerate(zip(x_train, y_train)):
            # Run the forward pass
            outputs = model(gaits)
            loss = criterion(outputs, labels)
            #loss_list.append(loss.item())

            # Backprop and perform Adam optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            # acc_list.append(correct / total)
            if (i + 1) % 1000 == 0:
                correct = 0.0
                model = model.eval()
                validation_loss = 0
                for (test_gait, test_lab) in zip(x_test, y_test):
                    prediction = model(test_gait)
                    validation_loss += criterion(prediction, test_lab)
                    if prediction.argmax() == test_lab:
                        correct += 1.0
                acc = correct / y_test.size()[0]
                acc_list.append(acc)
                validation_loss /= x_test.shape[0]
                loss_list.append(validation_loss)
                model = model.train()
                    
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, validation_loss,
                              acc * 100))
        
    correct = 0.0
    model = model.eval()
    validation_loss = 0
    for (test_gait, test_lab) in zip(x_test, y_test):
        prediction = model(test_gait)
        validation_loss += criterion(prediction, test_lab)
        if prediction.argmax() == test_lab:
            correct += 1.0
    acc = correct / y_test.size()[0]
    acc_list.append(acc)
    validation_loss /= x_test.shape[0]
    loss_list.append(validation_loss)
    model = model.train()
    return loss_list, acc_list
            

In [ ]:
gc = Gait_Classifier()
learning_rate = 3e-5
optimizer = torch.optim.Adam(gc.parameters(), lr=learning_rate)
(loss, acc) = train_model(gc, optimizer, gait_cycles_train, labels_train, gait_cycles_test, labels_test, 9)


In [ ]:
correct = 0.0
gc = gc.eval()
validation_loss = 0
for (batch_test_gaits, batch_test_labs) in zip(gait_cycles_train, labels_train):
    #print(batch_test_gaits.shape)
    for (test_gait, test_lab) in zip (batch_test_gaits, batch_test_labs):
        #print(test_gait.shape)
        prediction = gc(test_gait.unsqueeze(0))
        #validation_loss += criterion(prediction, test_lab)
        if prediction.argmax() == test_lab:
            correct += 1.0
acc = correct / (labels_train.size()[0]*9)
print(acc)
print(labels_train.size()[0])
#acc_list.append(acc)
#validation_loss /= x_test.shape[0]
#loss_list.append(validation_loss)
gc = gc.train()

In [ ]:
plt.plot(range(len(acc)),acc)
plt.title('Classification Accuracy as a Function of Epoch')
plt.ylabel('% Accuracy')
plt.xlabel("Epoch")
#plt.savefig("classification_accuracy_Test.png")
plt.show()

In [ ]:
(loss, acc) = train_model(gc, optimizer, gait_cycles_train, labels_train, gait_cycles_test, labels_test, 1, num_epochs=20)





In [ ]:
wrong_indices = []
for i in range(len(labels_test)):
    if gc(gait_cycles_test[i]).argmax() != labels_test[i]:
        wrong_indices.append(i)

gait_cycles_numpy = gait_cycles[-500:]
for i in wrong_indices:
    one_person = gait_cycles_numpy[i]
    t = range(one_person.shape[0])
    mag = one_person[:,3]
    plt.plot(t,mag)
    plt.show()

In [ ]:
print(labels_test[i])

In [ ]:
torch.save(gc.state_dict(), 'model_new.pt')

In [ ]:
gc = torch.load('model.pt')
gc.eval()